# Agente de Gestión de Inventario con MCP Local

## Descripción

En este ejercicio, construirás un **sistema completo de agentes con herramientas MCP locales** para gestión de inventario de productos, combinando un servidor MCP local con un agente cliente que responde preguntas sobre stock y ventas.

### Arquitectura MCP Local
```
Pregunta sobre inventario
    ↓
[InventoryAgent]
    ↓
[MCPStdioTool] → python server.py
    ↓
[Herramientas locales]
    ├→ get_inventory_levels()
    └→ get_weekly_sales()
    ↓
Análisis y respuesta
```

### Ventajas de MCP Local
- **Control total**: Datos bajo tu control, no en cloud
- **Rendimiento**: Latencia mínima para datos locales
- **Seguridad**: Sin exposición de datos en internet
- **FastMCP**: Framework ligero para crear servidores MCP

### Crear el servidor MCP local

Primero, creamos un archivo `server.py` que define las herramientas `get_inventory_levels` y `get_weekly_sales` usando `FastMCP`. Este archivo se ejecutará como un proceso separado.

In [ ]:
server_code = """
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("ProductInventory")

@mcp.tool()
def get_inventory_levels() -> dict:
    \"\"\"Devuelve el inventario actual de productos en el almacén.\"\"\"
    return {
        "Portátil HP EliteBook": 12,
        "Monitor Dell 27 pulgadas": 8,
        "Teclado mecánico": 25,
        "Ratón inalámbrico": 30,
        "Auriculares Bluetooth": 18,
        "Webcam HD": 15,
        "Silla ergonómica": 6,
        "Mesa ajustable": 4,
        "Lámpara LED escritorio": 22,
        "Hub USB-C": 35,
    }

@mcp.tool()
def get_weekly_sales() -> dict:
    \"\"\"Devuelve las ventas semanales de productos de la última semana.\"\"\"
    return {
        "Portátil HP EliteBook": 8,
        "Monitor Dell 27 pulgadas": 12,
        "Teclado mecánico": 15,
        "Ratón inalámbrico": 20,
        "Auriculares Bluetooth": 10,
        "Webcam HD": 7,
        "Silla ergonómica": 5,
        "Mesa ajustable": 3,
        "Lámpara LED escritorio": 9,
        "Hub USB-C": 18,
    }

if __name__ == "__main__":
    mcp.run()
"""

print("Creating local MCP server file...")
with open("local_MCP_server.py", "w") as f:
    f.write(server_code)
print("Server file 'local_MCP_server.py' created successfully")


### Cargar librerías y variables de entorno para el cliente

In [ ]:
import asyncio
import os
from dotenv import load_dotenv, find_dotenv
from agent_framework import ChatAgent, MCPStdioTool
from agent_framework.openai import OpenAIChatClient

load_dotenv(find_dotenv(usecwd=True))

# GitHub Models client configuration
MODEL_NAME = os.getenv("GITHUB_MODEL", "openai/gpt-4o")

### Crear y ejecutar el agente cliente

El `ChatAgent` se conecta al servidor MCP local usando `MCPStdioTool`, que inicia el proceso `python server.py`. Luego, podemos hacerle preguntas al agente sobre el inventario.

In [ ]:
async def query_inventory(prompt):
    print("Starting local MCP server...")
    
    # GitHub Models client configuration
    client = OpenAIChatClient(
        model_id=MODEL_NAME,
        api_key=os.environ["GITHUB_TOKEN"],
        base_url="https://models.github.ai/inference"
    )
    
    async with (
        MCPStdioTool(
            name="productinventory",
            command="python",
            args=["local_MCP_server.py"],  # Use file created in previous cell
        ) as mcp_server,
        ChatAgent(
            chat_client=client,
            name="InventoryAgent",
            instructions=(
                "Eres un asistente de gestión de inventario que puede consultar niveles de stock y ventas semanales de productos."
            ),
        ) as agent,
    ):
        print("Local MCP server connected")
        print(f"Running inventory query: {prompt}")
        result = await agent.run(prompt, tools=mcp_server)
        print("Query result:")
        print(result)


In [ ]:

# Enter your question here
print("Starting inventory analysis...")
user_prompt = "Lista los productos con inventario menor a 15 unidades y sus ventas semanales."
await query_inventory(user_prompt)
print("Inventory analysis completed")

In [ ]:

# Enter your question here
print("Starting inventory analysis...")
user_prompt = "¿Qué productos podrían agotarse esta semana considerando las ventas semanales y el inventario actual?"
await query_inventory(user_prompt)
print("Inventory analysis completed")